# Milestone 1: Environment Preparation (10 points by 10/30)


In [ ]:
# install nni
! npm install -g localtunnel
! pip install nni

# other dependencies
! pip install torch
import time

# because google colab is hosted on a google ip;
# we first need to download ngrok and unzip it
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

# create our project folder
! mkdir -p semseg_repo

# clone remote repository from repo we cloned from seed
! git clone https://github.com/SebastianAlcock/CS301101-Semantic-Segmentation-of-Satellite-Imagery semseg_repo/nni
# access ngrok with our personal authtoken
! ./ngrok authtoken 2Gova0foGFQhbYO3XBasI3LxD0P_5htGJmi8jFj2P6uf4VWaS

# launch the nni ui on a given port and url using the config file we produced
! nnictl create --config semseg_repo/nni/config.yml --port 2>&1 5000 &
time.sleep(10)
get_ipython().system_raw('lt --port 5000 >> url.txt 2>&1 &')
time.sleep(5)

# supplying the url so we can access it
! curl -s http://localhost:4040/api/tunnels
! cat url.txt

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 4.192s
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56.0 MB 27 kB/s 
     |████████████████████████████████| 106 kB 48.8 MB/s 
     |████████████████████████████████| 51 kB 7.2 MB/s 
     |████████████████████████████████| 130 kB 55.1 MB/s 
     |████████████████████████████████| 127 kB 56.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-10-30 00:45:55--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io 

In [ ]:
! nnictl stop --all

INFO:  Stopping experiment l1tdwfup
INFO:  Stop experiment success.
